<a href="https://colab.research.google.com/github/MovingJoon/Deeplearning-from-scratch/blob/master/Chapter_5%20%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None

    def forward(self,x,y):
        self.x=x
        self.y=y
        out=x*y

        return out

    def backward(self,dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

In [2]:
class Relu:
    def __int__(self):
        self.mask=None

    def forward(self,x):
        self.mask = (x<=0)
        out=x.copy()
        out[self.mask] = 0

In [3]:
#Chain rule
apple=100
apple_num=2
tax=1.1

#layer
mul_apple_layer=MulLayer()
mul_tax_layer=MulLayer()

#forward
apple_price=mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

#backward
dprice=1
dapples_price, dtax = mul_tax_layer.backward(dprice)
dapple_price,dapple_number=mul_apple_layer.backward(dapples_price)

print('dapples_price=', dapples_price,'dtax=',dtax, 'dapple_price=',dapple_price, 'dapple_number=',round(dapple_number))

dapples_price= 1.1 dtax= 200 dapple_price= 2.2 dapple_number= 110


In [4]:
import numpy as np
from functions import*
class Relu:
    def __init__(self):
        self.mask=None
        self.mask2=None
    def forward(self,x):
        self.mask = (x<=0) #boolean
        out = x.copy()
        out[self.mask] = 0 #false -> 0

        return out

    def backword(self,dout):
        self.mask2 = (dout>0)
        dout[self.mask2]= 1
        dx=dout

        return dx

ModuleNotFoundError: ignored

In [ ]:
class sigmoid:
    def __init__(self):
        self.out = None

    def forward(self,x):
        out = 1/(1+np.exe(-x))
        self.out = out

        return out

    def backward(self,dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b =b
        self.x = None
        self.dw = None
        self.db =None

    def forward(self,x):
        self.x= x
        out = np.dot(x,self.W)+self.b

        return out

    def backward(self, dout):
        dx= np.dot(dout,self.W.T)
        self.dw = np.dot(dout, self.x.T)
        self.db = np.sum(dout,axis=0)

        return dx
        
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None    #소프트맥스의 출력
        self.t = None    #정답레이블

    def forward(self,x,t):
        self.t =t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t) #손실함수
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]                   #배치사이즈
        dx = (self.y - self.t) / batch_size

        return dx


In [ ]:
import numpy as np
from layers import *
from gradient import numerical_gradient
from collections import OrderedDict


class TwolayerNet:
    def __init__(self, input_size,hidden_size ,output_size, weight_init_std=0.01):

        #가중치 초기화
        self.parmas = {}
        self.parmas['W1'] = weight_init_std * np.random.randn(input_size,output_size)
        self.parmas['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.parmas['b1'] = np.zeros(hidden_size)
        self.parmas['b2'] = np.zeors(output_size)

        #계층 만들기
        self.layers =OrderedDict()
        self.layers['Affine1'] = Affine(self.parmas['W1'],self.parmas['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.parmas['W2'],self.parmas['b2'])

        self.lastLayer = SoftmaxWithLoss()

   #순전파
    def predict(self,x):
        for layer in self.layers.values():
            x=layer.forward(x)

            return x

    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)
        if t.ndim!=1 : t = np.argmax(t,axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])

        return accuracy

    def gradient(self,x,t):
        #순전파
        self.loss(x,t)

        #역전파
        dout =1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW